In [ ]:
import sys; sys.path.append("..") 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from src.utility import loadfile, savefile, exists
from src.dataprocessing import *
from src import rnn

## load data

In [ ]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-5", "LT-3061-2"]
process_var = input_var + output_var

In [ ]:
min_len = 100
continuous_path = './data/3_continuous'

cts_list = []
i = 1
while exists(f"{continuous_path}/cts_{min_len}/dataset {min_len}_{i}.csv"):
    cts_df = loadfile(continuous_path, f"cts_{min_len}/dataset {min_len}_{i}", 'csv')
    cts_list.append(cts_df)
    i += 1

In [ ]:
target_list = cts_list

# history size and future size
history_list = [10, 20, 30, 40]
future_list = [10, 20, 30]
step = 1

# variable selection
history_var = process_var
future_var = output_var

history_num = len(history_var)
future_num = len(future_var)

# supervised attention factor
delta = 1
att_type = 'linear'
        
# test data split        
test_size = 0.2
test_num = -1

# model structure
num_layers = 1
num_neurons = 50
dense_layers = 1
dense_neurons = 50
model_type = 'seq2seq_gru'

iteration_list = [x for x in range(1,11)]
for iteration in iteration_list:
    for history_size in history_list:
        for future_size in future_list:
            print(f"{iteration}th iteration")
            print(f"history size: {history_size}")
            print(f"future size: {future_size}")
            history_series = []
            future_series = []

            # data to series
            for i in range(len(target_list)):
                history, future = data2series(target_list[i], history_size, history_var, future_size, future_var,
                                            step, start_idx=0, end_idx=None)
                if not i:
                    history_series = history
                    future_series = future
                else:
                    history_series = np.concatenate([history_series, history], axis=0)
                    future_series = np.concatenate([future_series, future], axis=0)
            
            factor = rnn.super_attention(delta, future_size, future_num, att_type)
            # Dual-attention Seq2Seq model
            DATT_seq2seq_GRU = rnn.RNN(history_series, history_var, future_series, future_var)
            # TT split
            DATT_seq2seq_GRU.train_test(test_size=test_size, test_num=test_num)
            # TV split
            valid_size = DATT_seq2seq_GRU.history_test.shape[0]/DATT_seq2seq_GRU.history_train.shape[0]
            DATT_seq2seq_GRU.train_valid(valid_size=valid_size)
            # scaling
            DATT_seq2seq_GRU.scaling()
            # modeling
            DATT_seq2seq_GRU.build_model(num_layers=num_layers, num_neurons=num_neurons, dense_layers=dense_layers, dense_neurons=dense_neurons, model_type=model_type, factor=factor)
            # training
            model_num = iteration
            model_name = f"{history_size}_{future_size}_{num_layers}_{num_neurons}_{dense_layers}_{dense_neurons}_{model_type}_{model_num}"
            if not exists(f"./model/{model_name}.h5"):
                DATT_seq2seq_GRU.train()
                DATT_seq2seq_GRU.save_model(f"./model/{model_name}", 'weights')
                
            else:
                DATT_seq2seq_GRU.model.load_weights(f"./model/{model_name}.h5")
            # test
            test_result = DATT_seq2seq_GRU.test()
            if not exists(f'./result/{model_name}.csv', ):
                savefile(test_result, './result', model_name)
            print("\n")
    print('\n\n')
print('end opitmization')